In [2]:
!pip install pyspark
!pip install kagglehub

In [3]:
import kagglehub
import pyspark
from pyspark.sql import SparkSession


In [4]:
spark = SparkSession.builder.appName('Ejercicio').getOrCreate()

In [5]:
spark

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mohitbansal31s/pokemon-dataset")

print("Path to dataset files:", path)

100%|██████████| 22.2k/22.2k [00:00<00:00, 9.07MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/mohitbansal31s/pokemon-dataset/versions/1


In [7]:
df_pokemon=spark.read.option('header','true').csv("/content/pokemon_data_pokeapi.csv")

df_pokemon.show(10)

+----------+--------------+-----+------+-------------------+----------+-----------+--------------------+----------+----------------+
|      Name|Pokedex Number|Type1| Type2|     Classification|Height (m)|Weight (kg)|           Abilities|Generation|Legendary Status|
+----------+--------------+-----+------+-------------------+----------+-----------+--------------------+----------+----------------+
| Bulbasaur|             1|Grass|Poison|       Seed Pokémon|       0.7|        6.9|Overgrow, Chlorop...|         1|              No|
|   Ivysaur|             2|Grass|Poison|       Seed Pokémon|       1.0|       13.0|Overgrow, Chlorop...|         1|              No|
|  Venusaur|             3|Grass|Poison|       Seed Pokémon|       2.0|      100.0|Overgrow, Chlorop...|         1|              No|
|Charmander|             4| Fire|  NULL|     Lizard Pokémon|       0.6|        8.5|  Blaze, Solar-power|         1|              No|
|Charmeleon|             5| Fire|  NULL|      Flame Pokémon|       1.

In [10]:
df_pokemon.printSchema()
df_pokemon_drop=df_pokemon.drop("Abilities","Type2")
df_pokemon_drop.show(5)
df_pokemon_drop.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Pokedex Number: string (nullable = true)
 |-- Type1: string (nullable = true)
 |-- Type2: string (nullable = true)
 |-- Classification: string (nullable = true)
 |-- Height (m): string (nullable = true)
 |-- Weight (kg): string (nullable = true)
 |-- Abilities: string (nullable = true)
 |-- Generation: string (nullable = true)
 |-- Legendary Status: string (nullable = true)

+----------+--------------+-----+--------------+----------+-----------+----------+----------------+
|      Name|Pokedex Number|Type1|Classification|Height (m)|Weight (kg)|Generation|Legendary Status|
+----------+--------------+-----+--------------+----------+-----------+----------+----------------+
| Bulbasaur|             1|Grass|  Seed Pokémon|       0.7|        6.9|         1|              No|
|   Ivysaur|             2|Grass|  Seed Pokémon|       1.0|       13.0|         1|              No|
|  Venusaur|             3|Grass|  Seed Pokémon|       2.0|      100.0|     

In [11]:
df_rename_columns=df_pokemon.withColumnRenamed("Type1","Type Pokemon")
df_rename_columns.show(5)

+----------+--------------+------------+------+--------------+----------+-----------+--------------------+----------+----------------+
|      Name|Pokedex Number|Type Pokemon| Type2|Classification|Height (m)|Weight (kg)|           Abilities|Generation|Legendary Status|
+----------+--------------+------------+------+--------------+----------+-----------+--------------------+----------+----------------+
| Bulbasaur|             1|       Grass|Poison|  Seed Pokémon|       0.7|        6.9|Overgrow, Chlorop...|         1|              No|
|   Ivysaur|             2|       Grass|Poison|  Seed Pokémon|       1.0|       13.0|Overgrow, Chlorop...|         1|              No|
|  Venusaur|             3|       Grass|Poison|  Seed Pokémon|       2.0|      100.0|Overgrow, Chlorop...|         1|              No|
|Charmander|             4|        Fire|  NULL|Lizard Pokémon|       0.6|        8.5|  Blaze, Solar-power|         1|              No|
|Charmeleon|             5|        Fire|  NULL| Flame P

In [13]:
df_select=df_rename_columns.select("Name","Type Pokemon","Generation")
df_select.show(5)

+----------+------------+----------+
|      Name|Type Pokemon|Generation|
+----------+------------+----------+
| Bulbasaur|       Grass|         1|
|   Ivysaur|       Grass|         1|
|  Venusaur|       Grass|         1|
|Charmander|        Fire|         1|
|Charmeleon|        Fire|         1|
+----------+------------+----------+
only showing top 5 rows



In [16]:
from ast import alias
from pyspark.sql.functions import count
df_tipos_pokemon=df_rename_columns.groupBy("Type Pokemon").agg(count("Name").alias("Total Pokemon"))
df_tipos_pokemon.show()

+------------+-------------+
|Type Pokemon|Total Pokemon|
+------------+-------------+
|       Water|          124|
|      Poison|           35|
|       Steel|           30|
|        Rock|           50|
|         Ice|           28|
|       Ghost|           31|
|       Fairy|           22|
|     Psychic|           58|
|      Dragon|           31|
|      Flying|            7|
|         Bug|           76|
|    Electric|           49|
|        Fire|           58|
|      Ground|           36|
|        Dark|           37|
|    Fighting|           37|
|       Grass|           86|
|      Normal|          110|
+------------+-------------+



In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

df_legendary = df_pokemon.filter(col("Legendary Status") == "Yes").select("Name")
df_legendary.show()


+---------+
|     Name|
+---------+
| Articuno|
|   Zapdos|
|  Moltres|
|   Mewtwo|
|      Mew|
|   Raikou|
|    Entei|
|  Suicune|
|    Lugia|
|    Ho-oh|
|   Celebi|
| Regirock|
|   Regice|
|Registeel|
|   Latias|
|   Latios|
|   Kyogre|
|  Groudon|
| Rayquaza|
|  Jirachi|
+---------+
only showing top 20 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

df_legendary = df_pokemon.filter(col("Generation") == 1).select("Name")
df_legendary.show()

+----------+
|      Name|
+----------+
| Bulbasaur|
|   Ivysaur|
|  Venusaur|
|Charmander|
|Charmeleon|
| Charizard|
|  Squirtle|
| Wartortle|
| Blastoise|
|  Caterpie|
|   Metapod|
|Butterfree|
|    Weedle|
|    Kakuna|
|  Beedrill|
|    Pidgey|
| Pidgeotto|
|   Pidgeot|
|   Rattata|
|  Raticate|
+----------+
only showing top 20 rows



In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

df_legendary = df_pokemon.filter(col("Legendary Status") == "Yes").select("Name")
df_legendary.show()

+---------+
|     Name|
+---------+
| Articuno|
|   Zapdos|
|  Moltres|
|   Mewtwo|
|      Mew|
|   Raikou|
|    Entei|
|  Suicune|
|    Lugia|
|    Ho-oh|
|   Celebi|
| Regirock|
|   Regice|
|Registeel|
|   Latias|
|   Latios|
|   Kyogre|
|  Groudon|
| Rayquaza|
|  Jirachi|
+---------+
only showing top 20 rows



In [19]:
df_legendary_gen = df_pokemon.filter(
    (col("Legendary Status") == "Yes") &
    (col("Generation").isin([1, 2, 3, 4]))
).select("Name", "Generation")

df_legendary_gen.show()

+---------+----------+
|     Name|Generation|
+---------+----------+
| Articuno|         1|
|   Zapdos|         1|
|  Moltres|         1|
|   Mewtwo|         1|
|      Mew|         1|
|   Raikou|         2|
|    Entei|         2|
|  Suicune|         2|
|    Lugia|         2|
|    Ho-oh|         2|
|   Celebi|         2|
| Regirock|         3|
|   Regice|         3|
|Registeel|         3|
|   Latias|         3|
|   Latios|         3|
|   Kyogre|         3|
|  Groudon|         3|
| Rayquaza|         3|
|  Jirachi|         3|
+---------+----------+
only showing top 20 rows



In [22]:
total_pokemon=df_pokemon.count()
print("Total de Pokemon:",total_pokemon)

Total de Pokemon: 905


In [23]:
df_pokemon.show()

+----------+--------------+------+------+-------------------+----------+-----------+--------------------+----------+----------------+
|      Name|Pokedex Number| Type1| Type2|     Classification|Height (m)|Weight (kg)|           Abilities|Generation|Legendary Status|
+----------+--------------+------+------+-------------------+----------+-----------+--------------------+----------+----------------+
| Bulbasaur|             1| Grass|Poison|       Seed Pokémon|       0.7|        6.9|Overgrow, Chlorop...|         1|              No|
|   Ivysaur|             2| Grass|Poison|       Seed Pokémon|       1.0|       13.0|Overgrow, Chlorop...|         1|              No|
|  Venusaur|             3| Grass|Poison|       Seed Pokémon|       2.0|      100.0|Overgrow, Chlorop...|         1|              No|
|Charmander|             4|  Fire|  NULL|     Lizard Pokémon|       0.6|        8.5|  Blaze, Solar-power|         1|              No|
|Charmeleon|             5|  Fire|  NULL|      Flame Pokémon| 

In [30]:
df_pokemon.printSchema()
df_rename_peso_altura=df_pokemon.withColumnRenamed("Weight (Kg)","Peso").withColumnRenamed("Height (m)","Altura")
df_rename_peso_altura.show()

root
 |-- Name: string (nullable = true)
 |-- Pokedex Number: string (nullable = true)
 |-- Type1: string (nullable = true)
 |-- Type2: string (nullable = true)
 |-- Classification: string (nullable = true)
 |-- Height (m): string (nullable = true)
 |-- Weight (kg): string (nullable = true)
 |-- Abilities: string (nullable = true)
 |-- Generation: string (nullable = true)
 |-- Legendary Status: string (nullable = true)

+----------+--------------+------+------+-------------------+------+-----+--------------------+----------+----------------+
|      Name|Pokedex Number| Type1| Type2|     Classification|Altura| Peso|           Abilities|Generation|Legendary Status|
+----------+--------------+------+------+-------------------+------+-----+--------------------+----------+----------------+
| Bulbasaur|             1| Grass|Poison|       Seed Pokémon|   0.7|  6.9|Overgrow, Chlorop...|         1|              No|
|   Ivysaur|             2| Grass|Poison|       Seed Pokémon|   1.0| 13.0|Overgr

In [33]:
df_float_pokemon = df_rename_peso_altura.withColumn("Altura_float", col("Altura").cast("float")).withColumn("Peso_float", col("Peso").cast("float"))
df_float_pokemon.show()

+----------+--------------+------+------+-------------------+------+-----+--------------------+----------+----------------+------------+----------+
|      Name|Pokedex Number| Type1| Type2|     Classification|Altura| Peso|           Abilities|Generation|Legendary Status|Altura_float|Peso_float|
+----------+--------------+------+------+-------------------+------+-----+--------------------+----------+----------------+------------+----------+
| Bulbasaur|             1| Grass|Poison|       Seed Pokémon|   0.7|  6.9|Overgrow, Chlorop...|         1|              No|         0.7|       6.9|
|   Ivysaur|             2| Grass|Poison|       Seed Pokémon|   1.0| 13.0|Overgrow, Chlorop...|         1|              No|         1.0|      13.0|
|  Venusaur|             3| Grass|Poison|       Seed Pokémon|   2.0|100.0|Overgrow, Chlorop...|         1|              No|         2.0|     100.0|
|Charmander|             4|  Fire|  NULL|     Lizard Pokémon|   0.6|  8.5|  Blaze, Solar-power|         1|      

In [38]:
from pyspark.sql.functions import col

# Pokémon con la altura mínima
pokemon_min = df_float_pokemon.orderBy(col("Altura_float").asc()).select("Name", "Altura_float").limit(5)
pokemon_min.show()

# Pokémon con la altura máxima
pokemon_max = df_float_pokemon.orderBy(col("Altura_float").desc()).select("Name", "Altura_float").limit(5)
pokemon_max.show()

+--------+------------+
|    Name|Altura_float|
+--------+------------+
|  Joltik|         0.1|
| Cosmoem|         0.1|
| Flabebe|         0.1|
|Cutiefly|         0.1|
|  Comfey|         0.1|
+--------+------------+

+----------+------------+
|      Name|Altura_float|
+----------+------------+
| Eternatus|        20.0|
|   Wailord|        14.5|
|   Steelix|         9.2|
|Celesteela|         9.2|
|      Onix|         8.8|
+----------+------------+

